# Zero-shot text classification with OpenAI's GPT models

This notebook illustrates how to use different GPT models provided by OpenAI for text classification.

In [1]:
import os
from openai import OpenAI

import re

In [2]:
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

## Define the task

In this example, we adapt the instruction for one of the tweet classification tasks examined in Gilardi et al. ([2023](https://www.pnas.org/doi/10.1073/pnas.2305016120)) "ChatGPT outperforms crowd workers for text-annotation tasks"

- see [this README file](https://github.com/haukelicht/llm_text_coding/data/gilardi_chatgpt_2023/README.md) for a description of the data and tasks covered in the paper
- see [this file](https://github.com/haukelicht/llm_text_coding/data/gilardi_chatgpt_2023/instructions.md) for a copy of their original task instructions

In [3]:
instructions = """
For each tweet in the sample, follow these instructions:

1. Carefully read the text of the tweet, paying close attention to details.
2. Classify the tweet as either relevant (1) or irrelevant (0)
"""

categories = ["Relevant", "Irrelevant"]

defintions = """
Tweets should be coded as RELEVANT when they directly relate to content moderation, as defined above. This includes tweets that discuss: social media platforms’ content moderation rules and practices, governments’ regulation of online content moderation, and/or mild forms of content moderation like flagging.
Tweets should be coded as IRRELEVANT if they do not refer to content moderation, as defined above, or if they are themselves examples of moderated content. This would include, for example, a Tweet by Donald Trump that Twitter has labeled as “disputed”, a tweet claiming that something is false, or a tweet containing sensitive content. Such tweets might be subject to content moderation, but are not discussing content moderation. Therefore, they should be coded as irrelevant for our purposes.
"""

## Example with text generation model

In [4]:

text = "@connybush Sorry hun, Ive removed the tags on IG d person handling my account thought you are my friend dats why u were tagged on both posts."

# clean the text 
text = re.sub(r'\s+', ' ', text).strip()

prompt = f"Classify the following text into one of the given categories: {categories}\n{defintions}\nOnly include the selected category in your response and no further text.\n\nText: {text}\n\nClassification:"

In [5]:
print(prompt)

Classify the following text into one of the given categories: ['Relevant', 'Irrelevant']

Tweets should be coded as RELEVANT when they directly relate to content moderation, as defined above. This includes tweets that discuss: social media platforms’ content moderation rules and practices, governments’ regulation of online content moderation, and/or mild forms of content moderation like flagging.
Tweets should be coded as IRRELEVANT if they do not refer to content moderation, as defined above, or if they are themselves examples of moderated content. This would include, for example, a Tweet by Donald Trump that Twitter has labeled as “disputed”, a tweet claiming that something is false, or a tweet containing sensitive content. Such tweets might be subject to content moderation, but are not discussing content moderation. Therefore, they should be coded as irrelevant for our purposes.

Only include the selected category in your response and no further text.

Text: @connybush Sorry hun, Iv

### Make the API Call

In [14]:
response = client.completions.create(
  model="davinci-002",
  prompt=prompt,
  max_tokens=2,
  top_p=1,
  temperature=0.0,
  seed=42,
  frequency_penalty=0,
  presence_penalty=0
)

### Parse the result

In [15]:
result = response.choices[0].text.strip()
result

'Relevant'

### Iterate over several examples

In [16]:
def classify_tweet(text):

  # clean the text 
  text = re.sub(r'\s+', ' ', text).strip()

  # construct the prompt
  prompt = f"Classify the following text into one of the given categories: {categories}\n{defintions}\nOnly include the selected category in your response and no further text.\n\nText: {text}\n\nClassification:"
  
  response = client.completions.create(
    model="davinci-002",
    prompt=prompt,
    max_tokens=2,
    top_p=1,
    temperature=0.0,
    seed=42,
    frequency_penalty=0,
    presence_penalty=0
  )
  
  result = response.choices[0].text.strip()
  
  return result

In [10]:
texts = [
    # negative examples ("irrelevant")
    "\"Turns out Mike Bloomberg is exactly what Elizabeth Warren needed to break through in the 2020 Democratic primary. And he’s not just a foil for her on the campaign trail — this is something she believes in, and it shows.\" https://t.co/1SyaHXrZlO",
    "@blackhat___05 ye raha new user name change kiya kamine ne😡🗡️😡🗡️😡🗡️😡 karo abhi FNfollow reopt aur block",
    "The Kid!\n \nRETWEET for a chance at a @RawlingsSports baseball signed by Ken Griffey Jr. and tune in to #Junior tonight at 8pm ET/5pm PT on MLB Network.\n \nRules: https://t.co/MdkXLh1CdN | NoPurNec, US 18+, Ends 6/22 https://t.co/8Xw0HpHz2G",
    "TW / gore \n\nif you come across an account and want to block them, make sure to cover the bottom half of your screen. the gore is normally at the bottom of the screen. again, stay safe, and take precaution",
    "@Godlesswh_re Blocked.  Is this another Nick account?",
    # positive examples ("relevant")
    "Twitter we want you to suspend Marcon's account.\n#twitterSuspendMacronAccount #TwitterSuspendMarcon @verified @Twitter @TwitterSupport",
    "Twitter needs to permanently suspend @realDonaldTrump account.  Who's with me?",
    "Toei is one of the most active reporters of content on Youtube and everything runs through an auto filter. Today, Toei dropped a ridiculous volume of their own series onto an official Youtube channel and GOT BANNED AND REPORTED BY THEMSELVES, TOEI.",
    "Marsha Blackburn: We Are Looking at Antitrust Laws and Section 230 on Tech Censorship https://t.co/lsOWzD0Yri",
    "#Facebook has banned the iconic photograph of a #Soviet solider waving the #USSR flag over the #Reichstag in May 1945. The social network claims the image violates its community guidelines for dangerous people and organizations...\n\nMORE: https://t.co/arpDN9Ss0P https://t.co/KGtGwE4D5J"
]

In [17]:
classifications = [classify_tweet(text) for text in texts]

In [18]:
classifications

['Relevant',
 'Relevant',
 'Relevant',
 'Relevant',
 'RELEV',
 'Relevant',
 'Relevant',
 'Relevant',
 'Relevant',
 'Relevant']

This doesn't look great =(

Let's try GPT 3.5 turbo and GPT 4 instead 👇

## With ChatGPT

In [32]:
def classify_tweet(text, model="gpt-3.5-turbo"):

  # clean the text 
  text = re.sub(r'\s+', ' ', text).strip()

  # construct input

  messages = [
    # system prompt
    {"role": "system", "content": f"Classify the following text into one of the given categories: {categories}\n{defintions}\nOnly include the selected category in your response and no further text."},
    # user input
    {"role": "user", "content": text},
  ]

  response = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=0.0,
    seed=42,
    frequency_penalty=0,
    presence_penalty=0
  )
  
  result = response.choices[0].message.content
  
  return result



### GPT 3.5 turbo

In [31]:
classifications = [classify_tweet(text) for text in texts]
classifications

['Irrelevant',
 'Irrelevant',
 'Irrelevant',
 'Irrelevant',
 'Irrelevant',
 'Relevant',
 'Relevant',
 'Irrelevant',
 'Relevant',
 'Relevant']

This looks already much better! 🥳

Performance: 

- 9 of 10 examples classified correctly (accuracy = 90%)
- 4 of 5 positive examples classified correctly (recall = 80%)
- 4 of 4 poisitve classifictions correct (precision = 100%)
- F1 score = 88.9%

### GPT 4

In [34]:
classifications_gpt4 = [classify_tweet(text, model='gpt-4') for text in texts]
classifications_gpt4

['Irrelevant',
 'Irrelevant',
 'Irrelevant',
 'Relevant',
 'Irrelevant',
 'Irrelevant',
 'Irrelevant',
 'Relevant',
 'Relevant',
 'Relevant']

## Multiple inputs per request

In theory, we can also combine several texts in one user message.

But as demonstrated below, this can cause problems, because classifications will depend on the order of texts in the input.

In [84]:
from typing import List

def classify_tweets(texts: List[str], model="gpt-3.5-turbo"):

  # clean the text 
  texts = [re.sub(r'\s+', ' ', text).strip() for text in texts]

  # construct input

  messages = [
    # system prompt (modified to handle multiple inputs)
    {"role": "system", "content": (
      "Act as a text classification system. "
      "Each line in the input is a separate tweet. "
      f"Classify each tweet into one of the given categories: {categories}\n{defintions}\n"
      "Only include the selected category in your response and no further text. "
      "Seperate the classifications of individual tweet by newline characters."
    )},
    # user input
    {"role": "user", "content": "\n".join(texts)},
  ]

  response = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=0.0,
    seed=42,
    frequency_penalty=0,
    presence_penalty=0
  )
  
  result = response.choices[0].message.content
  
  return result.split("\n")

In [86]:
classifications = classify_tweets(texts, model="gpt-4")
len(texts), len(classifications)

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Irrelevant\nIrrelevant\nIrrelevant\nRelevant\nRelevant\nRelevant\nRelevant\nRelevant\nRelevant\nRelevant', role='assistant', function_call=None, tool_calls=None))]


(10, 10)

In [91]:
from tqdm.auto import tqdm
# create a list of indexes from 0-9 and reshuffle it
import random
idxs = list(range(10))

# set the seed
random.seed(42)

results = []
n_iter = 5
for i in tqdm(range(n_iter), total=n_iter, desc="Iteration"):
    random.shuffle(idxs)
    inputs = [texts[i] for i in idxs]
    outputs = classify_tweets(inputs, model="gpt-4")
    sorted_outputs = [c for _, c in sorted(zip(idxs, outputs))]
    results.append(sorted_outputs)

Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Relevant\nIrrelevant\nIrrelevant\nRelevant\nIrrelevant\nIrrelevant\nRelevant\nIrrelevant\nIrrelevant\nIrrelevant', role='assistant', function_call=None, tool_calls=None))]
[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Relevant\nIrrelevant\nIrrelevant\nIrrelevant\nIrrelevant\nIrrelevant\nIrrelevant\nRelevant\nRelevant\nIrrelevant', role='assistant', function_call=None, tool_calls=None))]
[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Relevant\nIrrelevant\nRelevant\nIrrelevant\nRelevant\nIrrelevant\nRelevant\nIrrelevant\nRelevant\nRelevant', role='assistant', function_call=None, tool_calls=None))]
[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Irrelevant\nRelevant\nIrrelevant\nRelevant\nRelevant\nRelevant\nIrrelevant\nRelevant\nIrrelevant\nRelevant', role='ass

In [94]:
import pandas as pd

pd.DataFrame(results, columns=[f"text{i:02d}" for i , _ in enumerate(texts, start=1)])

,text01,text02,text03,text04,text05,text06,text07,text08,text09,text10
0,Irrelevant,Irrelevant,Irrelevant,Irrelevant,Irrelevant,Irrelevant,Irrelevant,Relevant,Relevant,Relevant
1,Irrelevant,Irrelevant,Irrelevant,Irrelevant,Irrelevant,Irrelevant,Irrelevant,Relevant,Relevant,Relevant
2,Irrelevant,Irrelevant,Irrelevant,Relevant,Irrelevant,Relevant,Relevant,Relevant,Relevant,Relevant
3,Irrelevant,Irrelevant,Irrelevant,Relevant,Irrelevant,Relevant,Relevant,Relevant,Relevant,Relevant
4,Irrelevant,Relevant,Irrelevant,Relevant,Relevant,Relevant,Relevant,Relevant,Relevant,Relevant


As you can see, the classifications of texts 2, 4, 5, 6, and 7 are not robust to the order of texts in the input 🤷‍♂️